In [17]:
import matplotlib.pyplot as plt
import vtk
import numpy as np
import sys
import math
import os
import glob
from vtk.util.numpy_support import *
import pandas
from multiprocessing import Pool
from scipy.stats import kurtosis, skew

In [27]:
def read_vti(filename):
    reader = vtk.vtkXMLImageDataReader()
    reader.SetFileName(filename)
    reader.Update()
    return reader.GetOutput()

def write_vti(filename,data):
    writer = vtk.vtkXMLImageDataWriter()
    writer.SetInputData(data)
    writer.SetFileName(filename)
    writer.Update()
    
def write_vtp(filename,data):
    writer = vtk.vtkXMLPolyDataWriter()
    writer.SetInputData(data)
    writer.SetFileName(filename)
    writer.Update()    
    
def read_plt(filename):
    reader = vtk.vtkAMReXParticlesReader()
    reader.SetPlotFileName(filename)
    reader.Update()
    return reader.GetOutput()

def compute_3d_to_1d_map(x,y,z,dimx,dimy,dimz):
    return x + dimx*(y+dimy*z)

In [59]:
input_classified_field_fname = '../out/mfix_local_grid/slic_compare_275.vti'
varname = 'moment_val' #feature_similarity, mixed_val

input_particle_data_fname = '/disk1/MFIX_fcc_plt/fcc27500/'
feature_threshold= 0.92 ## why this needs to be higher than the scalar field threshold?

nbins = [128,16,128]

In [60]:
## read the data from disk
classified_field = read_vti(input_classified_field_fname)
particle_data = read_plt(input_particle_data_fname)

numPieces = particle_data.GetBlock(0).GetNumberOfPieces()
print("total pieces:", numPieces)

## compute global bound
xmin = []
xmax = []
ymin = []
ymax = []
zmin = []
zmax = []
for i in range(numPieces):
    piece_bound = particle_data.GetBlock(0).GetPiece(i).GetBounds()
    xmin.append(piece_bound[0])
    xmax.append(piece_bound[1])
    ymin.append(piece_bound[2])
    ymax.append(piece_bound[3])
    zmin.append(piece_bound[4])
    zmax.append(piece_bound[5])

spatial_range_x = [np.min(xmin), np.max(xmax)]
spatial_range_y = [np.min(ymin), np.max(ymax)]
spatial_range_z = [np.min(zmin), np.max(zmax)]

print spatial_range_x, spatial_range_y, spatial_range_z

('total pieces:', 256)
[7.371875107241118e-05, 0.07861855596764976] [7.056479690719102e-05, 0.003128851708627044] [7.308852697741997e-05, 0.050727305546762115]


In [61]:
## Combine all the data points to one array
NBlocks = particle_data.GetNumberOfBlocks()
print("total blocks:",NBlocks)

mpData = particle_data.GetBlock(0)
numPieces = mpData.GetNumberOfPieces()
print("total pieces:", numPieces)

classified_array = classified_field.GetPointData().GetArray(varname)

filtered_particles = vtk.vtkPolyData()
filtered_pts_arr = vtk.vtkPoints()

for i in range(numPieces):
    data = mpData.GetPiece(i)
    
    if data is not None:
        allpts = data.GetPoints()

        for i in range(data.GetNumberOfPoints()):
            pts = allpts.GetPoint(i)

            xBinId = int((pts[0]-spatial_range_x[0])/(spatial_range_x[1]-spatial_range_x[0])*nbins[0])
            if xBinId==nbins[0]:
                xBinId=xBinId-1
            yBinId = int((pts[1]-spatial_range_y[0])/(spatial_range_y[1]-spatial_range_y[0])*nbins[1])
            if yBinId==nbins[1]:
                yBinId=yBinId-1
            zBinId = int((pts[2]-spatial_range_z[0])/(spatial_range_z[1]-spatial_range_z[0])*nbins[2])
            if zBinId==nbins[2]:
                zBinId=zBinId-1

            ## Get one-D ID
            oneD_idx = compute_3d_to_1d_map(xBinId,yBinId,zBinId,nbins[0],nbins[1],nbins[2])

            feature_value = classified_array.GetTuple1(oneD_idx)

            if feature_value > feature_threshold:
                filtered_pts_arr.InsertNextPoint(pts)

filtered_particles.SetPoints(filtered_pts_arr)   
print 'done'

('total blocks:', 1)
('total pieces:', 256)
done


In [63]:
write_vtp('/home/sdutta/Desktop/filtered_particles.vtp', filtered_particles)